In [ ]:
import holoviews as hv
import hvplot.pandas
import numpy as np
import pandas as pd

from holonote.annotate import Annotator, SQLiteDB
from holonote.annotate.annotator import Style

connector = SQLiteDB(table_name="test_curves")

speed_data = pd.read_parquet("./assets/example.parquet")
speed_data["TIME"] = np.arange(speed_data["TIME"].size)  # int for now
speed_curve = speed_data.hvplot("TIME", "SPEED")

In [ ]:
annotator = Annotator(
    speed_curve, fields=["description", "category"], connector=connector
)
# Sample data
data = {
    "category": pd.Categorical(["A", "B", "A", "C", "B"]),
    "start_number": np.array([1, 6, 11, 16, 21]) * 1000,
    "end_number": np.array([5, 10, 15, 20, 25]) * 1000,
    "description": list("ABCDE"),
}

annotator.define_annotations(pd.DataFrame(data), TIME=("start_number", "end_number"))
annotator.df

In [ ]:
import panel as pn

w1 = pn.widgets.MultiSelect(value=["A"], options=list("ABC"))
w2 = pn.widgets.Select(value="category", options=["description", "category"])
color_dim = hv.dim("category").categorize(
    categories={"B": "red", "A": "blue", "C": "green"}, default="grey"
)
# style.groupby = "category"  # if not widget was used
# style.visible = ["A", "B"]  # if not widget was used

# Better if I did something like this: Annotator(..., style=style), only here for convenience.
style = Style(visible=w1, groupby=w2, color=color_dim)
annotator.style = style
pn.Row(w1, w2)

In [ ]:
annotator * speed_curve

In [ ]:
style.alpha = 0.2
style.selection_alpha = 1

In [ ]:
# Not understanding why this is not the same as the color above... DynamicMap cache.
style.groupby = None
annotator * speed_curve

In [ ]:
style.edit_alpha = 1